In [1]:
import numpy as np 
import matplotlib.pyplot as plt
from copy import deepcopy
from collections import defaultdict

In [2]:
Cs = np.array([[0, 1, 1],
               [1, 0, 0],
               [1, 0, 0],
               [1, 0, 0],
               [0, 1, 1]])

Ct = np.array([[-1],
               [-1],
               [-1]])

Es = np.array([[1, 1, 1],
               [1, 0, 0],
               [1, 1, 0],
               [1, 0, 0],
               [1, 1, 1]])

Et = np.array([[-1],
               [-1],
               [1]])


Rs = np.array([[1, 1, 1],
               [1, 0, 1],
               [1, 1, 1],
               [1, 1, 0],
               [1, 0, 1]])

Rt = np.array([[-1],
               [1],
               [-1]])


inputs = [Cs, Es, Rs]
targets = [Ct, Et, Rt]

for i in range(len(inputs)):
    inputs[i] = np.where(inputs[i] == 0, -1, inputs[i])
    inputs[i] = inputs[i].reshape(inputs[i].shape[0] * inputs[i].shape[1], -1)
    targets[i] = np.where(targets[i] == 0, -1, targets[i])
    targets[i] = targets[i].reshape(targets[i].shape[0] * targets[i].shape[1], -1)

# الف و ب)

In [3]:
class BAM:
    '''
    Bidirectional Associative Memory
    '''
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets
        self.W = np.sum([s @ t.T for s, t in zip(inputs, targets)], axis=0)


    def run(self, x, y, n_iter=10):
        x_first = deepcopy(x)
        y_first = deepcopy(y)

        x_out = deepcopy(x)
        y_out = deepcopy(y)

        def active(in_i, out_i, threshold=0):
            if in_i > threshold:
                return 1
            elif in_i == threshold:
                return out_i
            elif in_i < threshold:
                return -1

        accs_x = []
        accs_y = []

        for it in range(n_iter):
            y_in = x_out.T @ self.W


            y_out = np.array([active(y_in.T[i], y_out[i]) for i in range(len(y_in.T))])
            y_out = y_out.reshape(y_out.shape[0], -1)

            x_in = y_out.T @ self.W.T
            x_out = np.array([active(x_in.T[i], x_out[i]) for i in range(len(x_in.T))])
            x_out = x_out.reshape(x_out.shape[0], -1)

            accs_x.append(np.sum(np.where(x_out == x_first, 1, 0)) / len(x_out))
            accs_y.append(np.sum(np.where(y_out == y_first, 1, 0)) / len(y_out))

            if accs_x[-1] == 1 and accs_y[-1] == 1:
                break

        return accs_x[-1], accs_y[-1]

bam = BAM(inputs, targets)

chars = ['C', "E", "R"]


for i, (s, t) in enumerate(zip(inputs, targets)):
    acc_x, acc_y = bam.run(s, t)
    print(f"{chars[i]}-input: {acc_x}")
    print(f"{chars[i]}-output: {acc_y}")


C-input: 1.0
C-output: 1.0
E-input: 0.8
E-output: 0.6666666666666666
R-input: 1.0
R-output: 1.0


 # پ)

In [5]:
def add_noise(inputs, noise=0.4):
    noise_inputs = deepcopy(inputs)
    for s in noise_inputs:
        for i in range(len(s)):
            for j in range(len(s[i])):
                if np.random.rand() < noise:
                    s[i][j] = -s[i][j]
    return noise_inputs

accs_y = defaultdict(list)

for _ in range(100):
    noise_inputs = add_noise(inputs)

    for i, (s, t) in enumerate(zip(noise_inputs, targets)):
        acc_x, acc_y = bam.run(s, t)
        accs_y[chars[i]].append(acc_y)

for i in range(len(chars)):
    print(f"{chars[i]}-output: {np.sum(np.where(np.array(accs_y[chars[i]]) == 1, 1, 0)) / float(len(accs_y[chars[i]]))}")
    print('-' * 20)


C-output: 0.64
--------------------
E-output: 0.0
--------------------
R-output: 0.31
--------------------


# ث)

In [6]:
Os = np.array([[0, 1, 0],
               [1, 0, 1],
               [1, 0, 1],
               [1, 0, 1],
               [0, 1, 0]])

Ot = np.array([[-1],
               [1],
               [1]])

Fs = np.array([[1, 1, 1],
               [1, 0, 0],
               [1, 1, 0],
               [1, 0, 0],
               [1, 0, 0]])

Ft = np.array([[1],
               [-1],
               [-1]])


Ps = np.array([[1, 1, 1],
               [1, 0, 1],
               [1, 1, 1],
               [1, 0, 0],
               [1, 0, 0]])

Pt = np.array([[1],
               [-1],
               [1]])



inputs.extend([Os, Fs, Ps])
targets.extend([Ot, Ft, Pt])

for i in range(3, len(inputs)):
    inputs[i] = np.where(inputs[i] == 0, -1, inputs[i])
    inputs[i] = inputs[i].reshape(inputs[i].shape[0] * inputs[i].shape[1], -1)
    targets[i] = np.where(targets[i] == 0, -1, targets[i])
    targets[i] = targets[i].reshape(targets[i].shape[0] * targets[i].shape[1], -1)

In [7]:
bam = BAM(inputs, targets)

chars = ["C", "E", "R", "O", "F", "P"]

for i, (s, t) in enumerate(zip(inputs, targets)):
    acc_x, acc_y = bam.run(s, t)
    print(f"{chars[i]}-input: {acc_x}")
    print(f"{chars[i]}-output: {acc_y}")

C-input: 1.0
C-output: 1.0
E-input: 1.0
E-output: 0.6666666666666666
R-input: 0.7333333333333333
R-output: 0.6666666666666666
O-input: 0.8666666666666667
O-output: 1.0
F-input: 1.0
F-output: 1.0
P-input: 0.8666666666666667
P-output: 0.6666666666666666
